In [1]:
import pandas as pd
import numpy as np
import re
from collections import Counter
import matplotlib.pyplot as plt
from wordsegment import load, segment
load()

In [2]:
# read all files
category_df = pd.read_csv('categories.csv')
full_df = pd.read_csv('full_data.csv')
outfit_df = pd.read_csv('outfits.csv')
tag_df = pd.read_csv('tagged_product_attributes.csv')
review_df = pd.read_csv('womens_clothing_reviews.csv')

In [3]:
category_df

,value,category,type,quantifier
0,Androgynous,style,general,many
1,Athleisure,style,general,many
2,Boho,style,general,many
3,Business Casual,style,general,many
4,Casual,style,general,many
...,...,...,...,...
898,Snakeskin,material,subcategory_scarves,many
899,Suede,material,subcategory_scarves,many
900,Synthetic,material,subcategory_scarves,many
901,Textile,material,subcategory_scarves,many


In [4]:
full_df

,product_id,brand,mpn,product_full_name,description,brand_category,created_at,updated_at,deleted_at,brand_canonical_url,details,labels,bc_product_id
0,01DSE9TC2DQXDG6GWKW9NMJ416,Banana Republic,514683,Ankle-Strap Pump,"A modern pump, in a rounded silhouette with an...",Unknown,2019-11-11 22:37:15.719107+00,2019-12-19 20:40:30.786144+00,NaN,https://bananarepublic.gap.com/browse/product....,"A modern pump, in a rounded silhouette with an...","{""Needs Review""}",NaN
1,01DSE9SKM19XNA6SJP36JZC065,Banana Republic,526676,Petite Tie-Neck Top,Dress it down with jeans and sneakers or dress...,Unknown,2019-11-11 22:36:50.682513+00,2019-12-19 20:40:30.786144+00,NaN,https://bananarepublic.gap.com/browse/product....,Dress it down with jeans and sneakers or dress...,"{""Needs Review""}",NaN
2,01DSJX8GD4DSAP76SPR85HRCMN,Loewe,4.001E+11,52MM Padded Leather Round Sunglasses,Padded leather covers classic round sunglasses.,JewelryAccessories/SunglassesReaders/RoundOval...,2019-11-13 17:33:59.581661+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/loewe-52mm-pad...,100% UV protection\nCase and cleaning cloth in...,"{""Needs Review""}",NaN
3,01DSJVKJNS6F4KQ1QM6YYK9AW2,Converse,4.00012E+11,Baby's & Little Kid's All-Star Two-Tone Mid-To...,The iconic mid-top design gets an added dose o...,"JustKids/Shoes/Baby024Months/BabyGirl,JustKids...",2019-11-13 17:05:05.203733+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/converse-babys...,Canvas upper\nRound toe\nLace-up vamp\nSmartFO...,"{""Needs Review""}",NaN
4,01DSK15ZD4D5A0QXA8NSD25YXE,Alexander McQueen,4.00011E+11,64MM Rimless Sunglasses,Hexagonal shades offer a rimless view with int...,JewelryAccessories/SunglassesReaders/RoundOval,2019-11-13 18:42:30.941321+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/alexander-mcqu...,100% UV protection\nGradient lenses\nAdjustabl...,"{""Needs Review""}",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
48974,01DSNVXY8EJ9FQAJ3MPDMPASHD,Bonpoint,4.00091E+11,Baby's Hooded Jacket,NaN,JustKids/Baby024months/InfantGirls/Outerwear,2019-11-14 21:08:28.040417+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/bonpoint-babys...,Cozy double breasted jacket crafted from cotto...,"{""Needs Review""}",NaN
48975,01DSGYHA3RMCHENBJVQPBGXM97,Laura Mercier,4.00096E+11,Flawless Fusion Ultra-Longwear Foundation,"WHAT IT ISA 15-hour long wearing, water resist...",SaksBeautyPlace/ForHer/Color/Foundation/Liquid...,2019-11-12 23:17:47.761072+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/laura-mercier-...,"WHAT IT ISA 15-hour long wearing, water resist...","{""Needs Review""}",NaN
48976,01DSJT8H12CAFQQH07SQSQWJ8C,Splendid,4.001E+11,Baby Girl's 2-Piece Ruffle Sweatshirt & Stripe...,Ruffled-trim sweatshirt lends romance to this ...,"JustKids/Baby024months/InfantGirls/Tops,JustKi...",2019-11-13 16:41:34.491443+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/splendid-baby-...,"Crewneck\nLong sleeves\nRib-knit neck, cuffs a...","{""Needs Review""}",NaN
48977,01DSH2PF9J7QZ44D842B3GMCFN,Florence Eiseman,4.00012E+11,Little Girl's Plaid & Velvet Dress,Pretty plaid dress with velvet collar and velv...,"JustKids/Girls214/ToddlerGirls24/Dresses,JustK...",2019-11-13 00:30:31.212215+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/florence-eisem...,Peter Pan collar\nShort sleeves\nBack zipper\n...,"{""Needs Review""}",NaN


In [5]:
outfit_df

,outfit_id,product_color_id,outfit_item_type
0,01DDBHC62ES5K80P0KYJ56AM2T,01DMBRYVA2S83FARJW4QWRBXNS,shoe
1,01DDBHC62ES5K80P0KYJ56AM2T,01DMBRYVA2V2Y8EN8YPY1E16EV,accessory1
2,01DDBHC62ES5K80P0KYJ56AM2T,01DMBRYVA23E5A8GBES8QJN3HF,top
3,01DDBHC62ES5K80P0KYJ56AM2T,01DMBRYVA2C7BQN0X7MNNYXBHW,bottom
4,01DMHCX50CFX5YNG99F3Y65GQW,01DMBRYVA2S83FARJW4QWRBXNS,shoe
...,...,...,...
5587,01E6MC52DNZE5AZ9MPZ2CYCXW9,01E4RW25YJBXGTTW54236ECJXQ,onepiece
5588,01E6MC52DPCTAAQVYNSKA7PER7,01E5ZS3RA3XV8JAN4R16HD1TA3,accessory1
5589,01E6MC52DPCTAAQVYNSKA7PER7,01E4RW25YJBXGTTW54236ECJXQ,onepiece
5590,01E6MC52DPCTAAQVYNSKA7PER7,01E2P0SJT1FXRYBGJCPF1P0XQ8,shoe


In [6]:
# have to do some cleaning/preprocessing
# since the attribute names and values are not completely identical 
# between the initial_tags and additional tags
tag_df

,product_id,product_color_id,attribute_name,attribute_value,file
0,01DVBTBPHR8WJTCVEN5AJRHF47,01DVBTBPJ41VVT00JJCG8TTZ2W,gender,Women,initial_tags
1,01DVA7QRXM928ZM0WWR7HFNTC1,01DVA7QRXXR9F0TWVE1HMC5ZQ3,Primary Color,Blacks,initial_tags
2,01DPGV4YRP3Z8J85DASGZ1Y99W,01DPGVGBK6YGNYGNF2S6FSH02T,style,Casual,initial_tags
3,01E1JM43NQ3H17PB22EV3074NX,01E1JM5WFWWCCCH3JTTTCYQCEQ,style,Modern,initial_tags
4,01DSE8Z2ZDAZKZ2SKCS1E3B3HK,01DSE8ZG8Y3FR8KWE2TY1QDWBF,shoe_width,Medium,initial_tags
...,...,...,...,...,...
119340,01DT2D3BN9K4SEH71MPB7WD5WA,01DT2D3BNFZBEA1BTZHSGCQN46,fit,relaxed,additional
119341,01DT2D3BN9K4SEH71MPB7WD5WA,01DT2D3BNFZBEA1BTZHSGCQN46,lengthblazerscoatsandjackets,midlengthathips,additional
119342,01DT2D3BN9K4SEH71MPB7WD5WA,01DT2D3BNFZBEA1BTZHSGCQN46,closureblazerscoatsandjackets,buttonedfront,additional
119343,01DT2D3BN9K4SEH71MPB7WD5WA,01DT2D3BNFZBEA1BTZHSGCQN46,occasion,work,additional


In [8]:
def seg(s):
    if not re.search(r'_', s):
        l = segment(s)
        if len(l) == 1:
            return l[0]
        else:
            return '_'.join(l)
    else:
        return s

tag_df['attribute_name_new'] = tag_df.attribute_name.apply(seg)
tag_df['attribute_value_new'] = tag_df.attribute_value.apply(seg)

In [9]:
sorted(tag_df.attribute_name_new.unique())

['additional_color',
 'belt_buckle_material',
 'belt_buckle_shape',
 'belt_closure',
 'belt_material',
 'belt_width',
 'calf_width',
 'category',
 'class_belts',
 'class_blazers_coats_and_jackets',
 'class_booties',
 'class_boots',
 'class_dress',
 'class_flats',
 'class_handbags',
 'class_jumpsuit_and_romper',
 'class_mules_and_slides',
 'class_pants_and_leggings',
 'class_pumps_and_heels',
 'class_sandals',
 'class_shorts',
 'class_skirts',
 'class_slippers',
 'class_sneakers_and_athletic',
 'class_sunglasses',
 'class_wedges',
 'closure_blazers_coats_and_jackets',
 'closure_handbag',
 'closure_one_piece',
 'closure_pants_and_leggings',
 'closure_shoe',
 'closure_shorts',
 'closure_skirts',
 'closure_sweater',
 'closure_top',
 'color',
 'dry_clean_only',
 'embellishment',
 'fit',
 'gender',
 'heel_height',
 'heel_shape',
 'leg_style',
 'leg_style_jeans',
 'length_blazers',
 'length_blazers_coats_and_jackets',
 'length_coats_and_jackets',
 'length_jeans',
 'length_one_piece',
 'length

In [10]:
sorted(tag_df.attribute_value_new.unique())

['100_cashmere',
 '100_cotton',
 '100linen',
 '100silk',
 '100wool',
 '14zip',
 '5pocketpantnondenim',
 'a_line',
 'abstract',
 'accessory',
 'acetate',
 'acid',
 'acrylic',
 'active',
 'active_track',
 'adjustable',
 'alpaca',
 'androgynous',
 'animal',
 'ankle',
 'ankle_strap',
 'anorak_raincoat_windbreaker',
 'asymmetrical',
 'athleisure',
 'aviator_square',
 'aviator_teardrop',
 'back_tie',
 'back_zip',
 'back_zipper',
 'backless',
 'backpacks',
 'baggy',
 'ballet',
 'band_collar',
 'beach_bags',
 'beaded',
 'beiges',
 'belt',
 'belt_bags_fannypack',
 'belted',
 'belts',
 'bermuda',
 'bike',
 'black',
 'blacks',
 'blazer',
 'blazer_dress',
 'blazers_coats_jackets',
 'block',
 'blouse',
 'blues',
 'boatneck',
 'body_con',
 'bodysuit',
 'boho',
 'bomber_varsity',
 'bootcut',
 'bootie',
 'booties',
 'boots',
 'bottom',
 'boyfriend',
 'brass',
 'browns',
 'bucket_bags',
 'buckle',
 'buckles',
 'burgundies',
 'business_casual',
 'business_casual_dress',
 'bustier',
 'button_down',
 'but

In [68]:
clean_tag_df = pd.DataFrame(tag_df[['product_id','attribute_name_new','attribute_value_new']]
                            .groupby(['attribute_name_new','product_id'])
                            .attribute_value_new
                            .apply(set)).reset_index().rename(columns={'attribute_name_new': 'category', 'attribute_value_new': 'value'})
clean_tag_df

,category,product_id,value
0,additional_color,01DPCDEF6SYX2E1NT5X7HJBFGY,{pinks}
1,additional_color,01DPCG1C1P0MQAV9NMS3N1TDAA,"{pinks, greens, blacks}"
2,additional_color,01DPCQ0CNKKHD3899ZEY9SEDHA,{whites}
3,additional_color,01DPCYE6AKJFKJSNCBDTQFG52Y,"{light_brown, grays, whites}"
4,additional_color,01DPD2FGA0FWFXJCKMT5QBJYHK,"{pinks, blacks, whites, blues, greens}"
...,...,...,...
56516,wash,01E606C7SRDEHGS8XZT4ZCNN6Y,{dark_blue}
56517,wash,01E606HQS50FZF40JZD9GDKRQ5,{black}
56518,wash,01E6070892T9HB6NZ6APHTZ70N,{black}
56519,wash,01E60710RJME1ADG2P26BGBQJ2,{dark_blue}


# Decide which N – 2 categories your group would like to focus on. 

You are required to focus on the proprietary attributes – style and occasion. Beyond that, your group should pick N - 2 other groups to analyze.

Examples of categories:
- Embellishments
- Category
- Prints
- Material

In [92]:
Counter(clean_tag_df.category).most_common(10)

[('category', 3969),
 ('primary_color', 3930),
 ('style', 3916),
 ('occasion', 3914),
 ('gender', 3913),
 ('fit', 2949),
 ('sizing', 2941),
 ('material_clothing', 2924),
 ('dry_clean_only', 2776),
 ('sleeve_length', 2047)]

In [125]:
attribute_list = ['category','fit','style','occasion']
for i in attribute_list:
    print(i)
    print(Counter(clean_tag_df[clean_tag_df.category==i].value.apply(tuple)).most_common(),'\n')

category
[(('top',), 963), (('bottom',), 899), (('shoe',), 672), (('one_piece',), 427), (('accessory',), 310), (('sweater',), 293), (('blazers_coats_jackets',), 280), (('sweatshirt_hoodie',), 118), (('top', 'accessory'), 3), (('bottom', 'top'), 1), (('blazers_coats_jackets', 'top'), 1), (('blazers_coats_jackets', 'sweater'), 1), (('top', 'sweatshirt_hoodie'), 1)] 

fit
[(('relaxed',), 1093), (('semi_fitted',), 654), (('straight_regular',), 553), (('fitted_tailored',), 455), (('oversized',), 105), (('semi_fitted', 'relaxed'), 34), (('relaxed', 'straight_regular'), 20), (('relaxed', 'oversized'), 12), (('semi_fitted', 'fitted_tailored'), 10), (('semi_fitted', 'straight_regular'), 7), (('semi_fitted', 'oversized'), 3), (('semi_fitted', 'relaxed', 'straight_regular'), 2), (('fitted_tailored', 'relaxed'), 1)] 

style
[(('casual', 'classic'), 353), (('modern', 'casual', 'classic'), 165), (('casual',), 161), (('business_casual', 'casual'), 152), (('modern', 'casual'), 147), (('business_casual

In [11]:
review_df

,clothing_id,age,title,review_text,rating,recommend,likes,division,department,class
0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses
...,...,...,...,...,...,...,...,...,...,...
23481,1104,34,Great dress for many occasions,I was very happy to snag this dress at such a ...,5,1,0,General Petite,Dresses,Dresses
23482,862,48,Wish it was made of cotton,"It reminds me of maternity clothes. soft, stre...",3,1,0,General Petite,Tops,Knits
23483,1104,31,"Cute, but see through","This fit well, but the top was very see throug...",3,0,1,General Petite,Dresses,Dresses
23484,1084,28,"Very cute dress, perfect for summer parties an...",I bought this dress for a wedding i have this ...,3,1,2,General,Dresses,Dresses


# Join the tagged_product_attributes table with full_data and investigate the details, descriptions, and tags used for each category

your goal is to get a sense for the business logic and rules used in tagging a certain product category.

In [101]:
df = pd.merge(full_df, clean_tag_df, on = 'product_id')
df = df.drop(['mpn','created_at','updated_at','deleted_at','bc_product_id','labels'], axis=1)
df

,product_id,brand,product_full_name,description,brand_category,brand_canonical_url,details,category,value
0,01E5ZXP5H0BTEZT9QD2HRZJ47A,A.L.C.,Lennox High Waist Cotton & Linen Pants,High-rise trousers tailored from a cool Italia...,Unknown,https://shop.nordstrom.com/s/a-l-c-lennox-high...,"True to size. High rise.\n31"" inseam; 14"" leg ...",category,{bottom}
1,01E5ZXP5H0BTEZT9QD2HRZJ47A,A.L.C.,Lennox High Waist Cotton & Linen Pants,High-rise trousers tailored from a cool Italia...,Unknown,https://shop.nordstrom.com/s/a-l-c-lennox-high...,"True to size. High rise.\n31"" inseam; 14"" leg ...",class_pants_and_leggings,{5pocketpantnondenim}
2,01E5ZXP5H0BTEZT9QD2HRZJ47A,A.L.C.,Lennox High Waist Cotton & Linen Pants,High-rise trousers tailored from a cool Italia...,Unknown,https://shop.nordstrom.com/s/a-l-c-lennox-high...,"True to size. High rise.\n31"" inseam; 14"" leg ...",closure_pants_and_leggings,{zip_fly_with_hook}
3,01E5ZXP5H0BTEZT9QD2HRZJ47A,A.L.C.,Lennox High Waist Cotton & Linen Pants,High-rise trousers tailored from a cool Italia...,Unknown,https://shop.nordstrom.com/s/a-l-c-lennox-high...,"True to size. High rise.\n31"" inseam; 14"" leg ...",dry_clean_only,{yes}
4,01E5ZXP5H0BTEZT9QD2HRZJ47A,A.L.C.,Lennox High Waist Cotton & Linen Pants,High-rise trousers tailored from a cool Italia...,Unknown,https://shop.nordstrom.com/s/a-l-c-lennox-high...,"True to size. High rise.\n31"" inseam; 14"" leg ...",fit,{semi_fitted}
...,...,...,...,...,...,...,...,...,...
68765,01E2KY2T6YV7R4VXT4BVV4TWT5,ATM ANTHONY THOMAS MELILLO,Stretch Pima Cotton Baby Tee,"Made from ATM’s signature Peruvian jersey, thi...",Unknown,https://shop.nordstrom.com/s/atm-anthony-thoma...,"True to size. XS=0, S=2-4, M=6-8, L=10.",sheer,{semi_sheer}
68766,01E2KY2T6YV7R4VXT4BVV4TWT5,ATM ANTHONY THOMAS MELILLO,Stretch Pima Cotton Baby Tee,"Made from ATM’s signature Peruvian jersey, thi...",Unknown,https://shop.nordstrom.com/s/atm-anthony-thoma...,"True to size. XS=0, S=2-4, M=6-8, L=10.",sizing,{regular}
68767,01E2KY2T6YV7R4VXT4BVV4TWT5,ATM ANTHONY THOMAS MELILLO,Stretch Pima Cotton Baby Tee,"Made from ATM’s signature Peruvian jersey, thi...",Unknown,https://shop.nordstrom.com/s/atm-anthony-thoma...,"True to size. XS=0, S=2-4, M=6-8, L=10.",sleeve_length,{short_sleeve}
68768,01E2KY2T6YV7R4VXT4BVV4TWT5,ATM ANTHONY THOMAS MELILLO,Stretch Pima Cotton Baby Tee,"Made from ATM’s signature Peruvian jersey, thi...",Unknown,https://shop.nordstrom.com/s/atm-anthony-thoma...,"True to size. XS=0, S=2-4, M=6-8, L=10.",style,"{casual, classic}"


# Build a model that will takes as input:
- product description (if any)
- product name
- product details (if any)
- brand

# And outputs the predicted attributes of this product.

For example, if the category you are using is fit,

INPUT:
- description: Blush linen Button fastenings along front 100% linen; lining: 100% cotton Dry clean Designer color: Shell Imported
- brand: Zimmermann
- brand_category: Clothing / Jumpsuits / Full Length

The actual clothing’s product URL is here.

OUTPUT:
- predicted fit: RELAXED

# style

In [116]:
style_df = df[df.category=='style'].copy()
style_list = style_df.value.apply(list).explode().unique()
for i in style_list:
    style_df[i] = style_df.value.apply(lambda x: 1 if i in x else 0)

style_df

,product_id,brand,product_full_name,description,brand_category,brand_canonical_url,details,category,value,modern,business_casual,classic,casual,androgynous,boho,retro,edgy,glam,romantic,athleisure
13,01E5ZXP5H0BTEZT9QD2HRZJ47A,A.L.C.,Lennox High Waist Cotton & Linen Pants,High-rise trousers tailored from a cool Italia...,Unknown,https://shop.nordstrom.com/s/a-l-c-lennox-high...,"True to size. High rise.\n31"" inseam; 14"" leg ...",style,"{modern, business_casual, classic}",1,1,1,0,0,0,0,0,0,0,0
26,01DSECZPAGJJC1EDC79JRBF4WK,Banana Republic,Mock-Neck Sweater Top,"Designed to be worn with high-waisted bottoms,...",Unknown,https://bananarepublic.gap.com/browse/product....,"Designed to be worn with high-waisted bottoms,...",style,"{business_casual, classic}",0,1,1,0,0,0,0,0,0,0,0
39,01DSECZPAGJJC1EDC79JRBF4WK,Banana Republic,Mock-Neck Sweater Top,"Designed to be worn with high-waisted bottoms,...",Unknown,https://bananarepublic.gap.com/browse/product....,"Designed to be worn with high-waisted bottoms,...",style,"{business_casual, classic}",0,1,1,0,0,0,0,0,0,0,0
53,01E607BHRQAJDZ76MJFN7RPRK1,Simon Miller,Rost Belted Shorts,Cinched at the natural waist and pleated for f...,Unknown,https://shop.nordstrom.com/s/simon-miller-rost...,"True to size. XS=0-2, S=4-6, M=6-8, L=8-10, XL...",style,"{modern, casual, androgynous}",1,0,0,1,1,0,0,0,0,0,0
69,01E5ZXJ6G03R7177X723CT04W0,A.L.C.,Minelli Silk Sleeveless Top,Painterly brushes of color that convey the flu...,Unknown,https://shop.nordstrom.com/s/a-l-c-minelli-sil...,"True to size.\n25 1/2"" length (size Medium)\nF...",style,"{modern, casual, boho}",1,0,0,1,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68710,01E2KZ045BCT1Q2Z2PBC0CWRGH,ATM ANTHONY THOMAS MELILLO,Micro Twill Pull On Pants,"These casual trousers are cut for an easy, dra...",Unknown,https://shop.nordstrom.com/s/atm-anthony-thoma...,"True to size. XS=0, S=2-4, M=6-8, L=10. High r...",style,"{casual, athleisure}",0,0,0,1,0,0,0,0,0,0,1
68725,01E2KXZ0WNQ96S64TDZE3WB7CX,ATM ANTHONY THOMAS MELILLO,Brushed Twill Crop Wide Leg Pants,Brushed twill gives these cropped wide-leg pan...,Unknown,https://shop.nordstrom.com/s/atm-anthony-thoma...,True to size. High rise.,style,"{modern, casual, business_casual, glam}",1,1,0,1,0,0,0,0,1,0,0
68739,01E2KXWWY6JAGENED7P62090NP,ATM ANTHONY THOMAS MELILLO,Slim Crop Pants,"Tailored from a soft, dense knit with cropped,...",Unknown,https://shop.nordstrom.com/s/atm-anthony-thoma...,Mid rise. True to size.,style,"{business_casual, classic, retro}",0,1,1,0,0,0,1,0,0,0,0
68754,01E2KZTEPE7QK1V8THASF7HWNR,ATM ANTHONY THOMAS MELILLO,Camo Print Silk Skirt,Flatlock-stitched bias seams enhance the fluid...,Unknown,https://shop.nordstrom.com/s/atm-anthony-thoma...,"True to size. XS=0, S=2-4, M=6-8, L=10.",style,"{casual, glam, retro}",0,0,0,1,0,0,1,0,1,0,0


In [115]:
style_df.drop(['category','value'],axis=1).to_csv('style_data.csv',index=False)

# occasion

In [118]:
occasion_df = df[df.category=='occasion'].copy()
occasion_list = occasion_df.value.apply(list).explode().unique()
for i in occasion_list:
    occasion_df[i] = occasion_df.value.apply(lambda x: 1 if i in x else 0)

occasion_df

,product_id,brand,product_full_name,description,brand_category,brand_canonical_url,details,category,value,work,day_tonight,vacation,weekend,night_out,cold_weather,workout
9,01E5ZXP5H0BTEZT9QD2HRZJ47A,A.L.C.,Lennox High Waist Cotton & Linen Pants,High-rise trousers tailored from a cool Italia...,Unknown,https://shop.nordstrom.com/s/a-l-c-lennox-high...,"True to size. High rise.\n31"" inseam; 14"" leg ...",occasion,{work},1,0,0,0,0,0,0
22,01DSECZPAGJJC1EDC79JRBF4WK,Banana Republic,Mock-Neck Sweater Top,"Designed to be worn with high-waisted bottoms,...",Unknown,https://bananarepublic.gap.com/browse/product....,"Designed to be worn with high-waisted bottoms,...",occasion,"{day_tonight, work}",1,1,0,0,0,0,0
35,01DSECZPAGJJC1EDC79JRBF4WK,Banana Republic,Mock-Neck Sweater Top,"Designed to be worn with high-waisted bottoms,...",Unknown,https://bananarepublic.gap.com/browse/product....,"Designed to be worn with high-waisted bottoms,...",occasion,"{day_tonight, work}",1,1,0,0,0,0,0
49,01E607BHRQAJDZ76MJFN7RPRK1,Simon Miller,Rost Belted Shorts,Cinched at the natural waist and pleated for f...,Unknown,https://shop.nordstrom.com/s/simon-miller-rost...,"True to size. XS=0-2, S=4-6, M=6-8, L=8-10, XL...",occasion,"{vacation, weekend}",0,0,1,1,0,0,0
64,01E5ZXJ6G03R7177X723CT04W0,A.L.C.,Minelli Silk Sleeveless Top,Painterly brushes of color that convey the flu...,Unknown,https://shop.nordstrom.com/s/a-l-c-minelli-sil...,"True to size.\n25 1/2"" length (size Medium)\nF...",occasion,"{day_tonight, vacation}",0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68706,01E2KZ045BCT1Q2Z2PBC0CWRGH,ATM ANTHONY THOMAS MELILLO,Micro Twill Pull On Pants,"These casual trousers are cut for an easy, dra...",Unknown,https://shop.nordstrom.com/s/atm-anthony-thoma...,"True to size. XS=0, S=2-4, M=6-8, L=10. High r...",occasion,{weekend},0,0,0,1,0,0,0
68721,01E2KXZ0WNQ96S64TDZE3WB7CX,ATM ANTHONY THOMAS MELILLO,Brushed Twill Crop Wide Leg Pants,Brushed twill gives these cropped wide-leg pan...,Unknown,https://shop.nordstrom.com/s/atm-anthony-thoma...,True to size. High rise.,occasion,"{work, weekend}",1,0,0,1,0,0,0
68735,01E2KXWWY6JAGENED7P62090NP,ATM ANTHONY THOMAS MELILLO,Slim Crop Pants,"Tailored from a soft, dense knit with cropped,...",Unknown,https://shop.nordstrom.com/s/atm-anthony-thoma...,Mid rise. True to size.,occasion,"{work, day_tonight}",1,1,0,0,0,0,0
68750,01E2KZTEPE7QK1V8THASF7HWNR,ATM ANTHONY THOMAS MELILLO,Camo Print Silk Skirt,Flatlock-stitched bias seams enhance the fluid...,Unknown,https://shop.nordstrom.com/s/atm-anthony-thoma...,"True to size. XS=0, S=2-4, M=6-8, L=10.",occasion,"{day_tonight, weekend}",0,1,0,1,0,0,0


In [119]:
occasion_df.drop(['category','value'],axis=1).to_csv('occasion_data.csv',index=False)

# fit

In [120]:
fit_df = df[df.category=='fit'].copy()
fit_list = fit_df.value.apply(list).explode().unique()
for i in fit_list:
    fit_df[i] = fit_df.value.apply(lambda x: 1 if i in x else 0)

fit_df

,product_id,brand,product_full_name,description,brand_category,brand_canonical_url,details,category,value,semi_fitted,straight_regular,oversized,relaxed,fitted_tailored
4,01E5ZXP5H0BTEZT9QD2HRZJ47A,A.L.C.,Lennox High Waist Cotton & Linen Pants,High-rise trousers tailored from a cool Italia...,Unknown,https://shop.nordstrom.com/s/a-l-c-lennox-high...,"True to size. High rise.\n31"" inseam; 14"" leg ...",fit,{semi_fitted},1,0,0,0,0
17,01DSECZPAGJJC1EDC79JRBF4WK,Banana Republic,Mock-Neck Sweater Top,"Designed to be worn with high-waisted bottoms,...",Unknown,https://bananarepublic.gap.com/browse/product....,"Designed to be worn with high-waisted bottoms,...",fit,{straight_regular},0,1,0,0,0
30,01DSECZPAGJJC1EDC79JRBF4WK,Banana Republic,Mock-Neck Sweater Top,"Designed to be worn with high-waisted bottoms,...",Unknown,https://bananarepublic.gap.com/browse/product....,"Designed to be worn with high-waisted bottoms,...",fit,{straight_regular},0,1,0,0,0
45,01E607BHRQAJDZ76MJFN7RPRK1,Simon Miller,Rost Belted Shorts,Cinched at the natural waist and pleated for f...,Unknown,https://shop.nordstrom.com/s/simon-miller-rost...,"True to size. XS=0-2, S=4-6, M=6-8, L=8-10, XL...",fit,{oversized},0,0,1,0,0
59,01E5ZXJ6G03R7177X723CT04W0,A.L.C.,Minelli Silk Sleeveless Top,Painterly brushes of color that convey the flu...,Unknown,https://shop.nordstrom.com/s/a-l-c-minelli-sil...,"True to size.\n25 1/2"" length (size Medium)\nF...",fit,{relaxed},0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68701,01E2KZ045BCT1Q2Z2PBC0CWRGH,ATM ANTHONY THOMAS MELILLO,Micro Twill Pull On Pants,"These casual trousers are cut for an easy, dra...",Unknown,https://shop.nordstrom.com/s/atm-anthony-thoma...,"True to size. XS=0, S=2-4, M=6-8, L=10. High r...",fit,{semi_fitted},1,0,0,0,0
68716,01E2KXZ0WNQ96S64TDZE3WB7CX,ATM ANTHONY THOMAS MELILLO,Brushed Twill Crop Wide Leg Pants,Brushed twill gives these cropped wide-leg pan...,Unknown,https://shop.nordstrom.com/s/atm-anthony-thoma...,True to size. High rise.,fit,{relaxed},0,0,0,1,0
68730,01E2KXWWY6JAGENED7P62090NP,ATM ANTHONY THOMAS MELILLO,Slim Crop Pants,"Tailored from a soft, dense knit with cropped,...",Unknown,https://shop.nordstrom.com/s/atm-anthony-thoma...,Mid rise. True to size.,fit,{fitted_tailored},0,0,0,0,1
68746,01E2KZTEPE7QK1V8THASF7HWNR,ATM ANTHONY THOMAS MELILLO,Camo Print Silk Skirt,Flatlock-stitched bias seams enhance the fluid...,Unknown,https://shop.nordstrom.com/s/atm-anthony-thoma...,"True to size. XS=0, S=2-4, M=6-8, L=10.",fit,{relaxed},0,0,0,1,0


In [128]:
fit_df.drop(['category','value'],axis=1).to_csv('fit_data.csv',index=False)

# category

In [129]:
category_df = df[df.category=='category'].copy()
category_list = category_df.value.apply(list).explode().unique()
for i in category_list:
    category_df[i] = category_df.value.apply(lambda x: 1 if i in x else 0)

category_df

,product_id,brand,product_full_name,description,brand_category,brand_canonical_url,details,category,value,bottom,top,sweater,one_piece,blazers_coats_jackets,shoe,accessory,sweatshirt_hoodie
0,01E5ZXP5H0BTEZT9QD2HRZJ47A,A.L.C.,Lennox High Waist Cotton & Linen Pants,High-rise trousers tailored from a cool Italia...,Unknown,https://shop.nordstrom.com/s/a-l-c-lennox-high...,"True to size. High rise.\n31"" inseam; 14"" leg ...",category,{bottom},1,0,0,0,0,0,0,0
15,01DSECZPAGJJC1EDC79JRBF4WK,Banana Republic,Mock-Neck Sweater Top,"Designed to be worn with high-waisted bottoms,...",Unknown,https://bananarepublic.gap.com/browse/product....,"Designed to be worn with high-waisted bottoms,...",category,{top},0,1,0,0,0,0,0,0
28,01DSECZPAGJJC1EDC79JRBF4WK,Banana Republic,Mock-Neck Sweater Top,"Designed to be worn with high-waisted bottoms,...",Unknown,https://bananarepublic.gap.com/browse/product....,"Designed to be worn with high-waisted bottoms,...",category,{top},0,1,0,0,0,0,0,0
41,01E607BHRQAJDZ76MJFN7RPRK1,Simon Miller,Rost Belted Shorts,Cinched at the natural waist and pleated for f...,Unknown,https://shop.nordstrom.com/s/simon-miller-rost...,"True to size. XS=0-2, S=4-6, M=6-8, L=8-10, XL...",category,{bottom},1,0,0,0,0,0,0,0
56,01E5ZXJ6G03R7177X723CT04W0,A.L.C.,Minelli Silk Sleeveless Top,Painterly brushes of color that convey the flu...,Unknown,https://shop.nordstrom.com/s/a-l-c-minelli-sil...,"True to size.\n25 1/2"" length (size Medium)\nF...",category,{top},0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68698,01E2KZ045BCT1Q2Z2PBC0CWRGH,ATM ANTHONY THOMAS MELILLO,Micro Twill Pull On Pants,"These casual trousers are cut for an easy, dra...",Unknown,https://shop.nordstrom.com/s/atm-anthony-thoma...,"True to size. XS=0, S=2-4, M=6-8, L=10. High r...",category,{bottom},1,0,0,0,0,0,0,0
68712,01E2KXZ0WNQ96S64TDZE3WB7CX,ATM ANTHONY THOMAS MELILLO,Brushed Twill Crop Wide Leg Pants,Brushed twill gives these cropped wide-leg pan...,Unknown,https://shop.nordstrom.com/s/atm-anthony-thoma...,True to size. High rise.,category,{bottom},1,0,0,0,0,0,0,0
68727,01E2KXWWY6JAGENED7P62090NP,ATM ANTHONY THOMAS MELILLO,Slim Crop Pants,"Tailored from a soft, dense knit with cropped,...",Unknown,https://shop.nordstrom.com/s/atm-anthony-thoma...,Mid rise. True to size.,category,{bottom},1,0,0,0,0,0,0,0
68742,01E2KZTEPE7QK1V8THASF7HWNR,ATM ANTHONY THOMAS MELILLO,Camo Print Silk Skirt,Flatlock-stitched bias seams enhance the fluid...,Unknown,https://shop.nordstrom.com/s/atm-anthony-thoma...,"True to size. XS=0, S=2-4, M=6-8, L=10.",category,{bottom},1,0,0,0,0,0,0,0


In [130]:
category_df.drop(['category','value'],axis=1).to_csv('category_data.csv',index=False)